In [16]:
import numpy as np 
import pandas as pd
import json
import math
import os
import warnings
warnings.filterwarnings('ignore')

In [17]:
path=os.getcwd()
path

'C:\\Users\\jayam\\Downloads\\7879phd'

In [18]:
train=pd.read_csv(path +"\\train_data.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])

In [19]:
train.shape

(22624345, 6)

### Checking duplicates in train data

In [20]:
trainRowsDF = train[train.duplicated()]
trainRowsDF.shape

(1827, 6)

#### There are 1827 duplicates in train data 

In [21]:
trainRowsDF.nunique() # so we have duplicates for 1 day

year           1
month          1
day            1
city          10
medicine    1288
sales         41
dtype: int64

In [22]:
trainRowsDF["year"].value_counts(),trainRowsDF["month"].value_counts(),trainRowsDF["day"].value_counts()

(2018    1827
 Name: year, dtype: int64,
 2    1827
 Name: month, dtype: int64,
 28    1827
 Name: day, dtype: int64)

#### We have duplicates on 2018 feb 28th

In [23]:
train.drop_duplicates().shape

(22622518, 6)

In [24]:
train.shape[0]-train.drop_duplicates().shape[0] # 22624345-22622518=1827 number  of duplicates

1827

#### After dropping duplicates we should have only  1 sales row for each day w.r.t each city and medicine, so let us do group by and find the shape  

In [25]:
train.groupby(["year","month","day","city","medicine"])["sales"].sum().reset_index().shape

(22601741, 6)

In [26]:
train.drop_duplicates().shape[0]-train.groupby(["year","month","day","city","medicine"])["sales"].sum().reset_index().shape[0]

20777

#### But  after doing groupby we got 22601741 number of rows which is 20777 rows less than the shape after dropping duplicates, so it indicates there are 2 sales for some days in the train data

#### Let us analyze what happened on that day i.e, on 28th FEB 2018

#### We know that we have duplicates on 28th FEB 2018 , so let us drop that day and check the shape of train data and compare with groupby fuction which only return one row per day  

In [27]:
train[(train["year"]!=2018)&(train["month"]!=2)&(train["day"]!=28)].shape

(16473331, 6)

In [28]:
train[(train["year"]!=2018)&(train["month"]!=2)&(train["day"]!=28)].groupby(["year","month","day","city","medicine"])["sales"].sum().reset_index().shape

(16473331, 6)

#### We have got same shape even after doing group by and without group excluding 28TH FEB 2018

#### So except on 28th Feb 2018 we dont have 2 sales registered on any other day, so let us analyse what happened on that day for any medicine 

In [29]:
train[(train["year"]==2018)&(train["month"]==2)&(train["day"]==28)& (train["medicine"]==1)]

,year,month,day,city,medicine,sales
19589366,2018,2,28,1,1,8.0
19591895,2018,2,28,2,1,4.0
19594278,2018,2,28,3,1,8.0
19596740,2018,2,28,4,1,32.0
19599420,2018,2,28,5,1,40.0
19602082,2018,2,28,6,1,40.0
19604698,2018,2,28,7,1,24.0
19607313,2018,2,28,8,1,44.0
19609864,2018,2,28,9,1,4.0
19612417,2018,2,28,10,1,8.0


#### As we thought we have 2 sales registered on that day except for 2,9 and 10 cities , let us check discount data , if this is the same case

In [30]:
discount=pd.read_csv(path +"\\discount_features.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])
discount.shape ### checking shape

(17277949, 4)

In [31]:
duplicateRowsDF = discount[discount.duplicated()]
duplicateRowsDF.shape # checking shape of duplicate data

(22009, 4)

In [32]:
duplicateRowsDF.nunique()# so we have duplicates for 1 day

date             1
city            10
medicine      2869
discounted       2
dtype: int64

In [33]:
duplicateRowsDF["date"].value_counts() # we have the same day 28th FEB 2018

2018-02-28    22009
Name: date, dtype: int64

In [34]:
discount["date"]=pd.to_datetime(discount["date"])# converting to date format

In [35]:
discount.drop_duplicates().shape

(17255940, 4)

In [36]:
discount.shape[0]-discount.drop_duplicates().shape[0] # 17277949-17255940=22009 number  of duplicates

22009

##### Discount shape is not matching with train data shape, let us check the head of discount

In [37]:
discount.head(2)

,date,city,medicine,discounted
0,2016-04-01,1,1,0
1,2016-04-01,1,2,0


#### Discount data starts from 1st april of 2016 

In [38]:
train.head(2)## where as train data starts from 2nd jan 2015

,year,month,day,city,medicine,sales
0,2015,1,2,1,1,24.0
1,2015,1,2,1,2,144.0


#### We dont have discounts data for 1 year 4 months of sales 

In [39]:
discount.tail(2)# so discount data ends at from 31st july of 2018

,date,city,medicine,discounted
17277947,2018-07-31,10,3165,0
17277948,2018-07-31,10,3057,0


In [40]:
train.tail(2) ## where as train data ends at 30th june 2018, so we have discount data available for our test predictions

,year,month,day,city,medicine,sales
22624343,2018,6,30,10,3157,4.0
22624344,2018,6,30,10,3165,56.0


#### Let us repeat the same process for discount data

In [41]:
discount.groupby(["date","city","medicine"])["discounted"].sum().reset_index().shape

(17255345, 4)

In [42]:
discount.drop_duplicates().shape

(17255940, 4)

In [43]:
discount.drop_duplicates().shape[0]-discount.groupby(["date","city","medicine"])["discounted"].sum().reset_index().shape[0]

595

#### After doing groupby we got 17255345 number of rows which is 595 rows less than the shape after dropping duplicates, so it indicates there are 2 discounts for some days in the discount data too

#### We know that we have duplicates on 28th FEB 2018 , so let us drop that day and check the shape of discount data and compare with groupby fuction which only return one row per day  

In [44]:
discount[(discount["date"]!="2018-2-28")].shape

(17228342, 4)

In [45]:
discount[(discount["date"]!="2018-2-28")].groupby(["date","city","medicine"])["discounted"].sum().reset_index().shape

(17228342, 4)

#### We have got same shape even after doing group by and without groupby excluding 28TH FEB 2018

#### Except on 28th Feb 2018 we dont have 2 disocunts registered on any other day, so let us analyse what happened on that day for any medicine 

In [46]:
discount[(discount["date"]=="2018-2-28")& (discount["medicine"]==1)]

,date,city,medicine,discounted
13469435,2018-02-28,1,1,0
13471964,2018-02-28,2,1,0
13474347,2018-02-28,3,1,0
13476809,2018-02-28,4,1,0
13479489,2018-02-28,5,1,0
13482151,2018-02-28,6,1,0
13484767,2018-02-28,7,1,0
13487382,2018-02-28,8,1,0
13489933,2018-02-28,9,1,0
13492486,2018-02-28,10,1,0


#### We could exactly observe the same pattern on the train data too as we thought we have 2 discounts registered on that day except for 2,9 and 10 cities

In [47]:
discount.discounted.value_counts(normalize=True)*100

0    95.375325
1     4.624675
Name: discounted, dtype: float64

#### We could see discounts are given for very less number of sales 

#### Converting to categorical columns to find some insights

In [48]:
train["medicine"]=train["medicine"].astype("category")
train["day"]=train["day"].astype("category")
train["city"]=train["city"].astype("category")
train["month"]=train["month"].astype("category")
train["year"]=train["year"].astype("category")

In [49]:
train.describe(include="all") ## using describe to find some insights

,year,month,day,city,medicine,sales
count,22624345.0,22624345.0,22624345.0,22624345.0,22624345.0,2.262434e+07
unique,4.0,12.0,31.0,10.0,3389.0,NaN
top,2017.0,3.0,28.0,4.0,453.0,NaN
freq,7330001.0,2210126.0,767353.0,2377044.0,12699.0,NaN
mean,NaN,NaN,NaN,NaN,NaN,5.166032e+01
std,NaN,NaN,NaN,NaN,NaN,1.100297e+02
min,NaN,NaN,NaN,NaN,NaN,-9.948000e+03
25%,NaN,NaN,NaN,NaN,NaN,1.200000e+01
50%,NaN,NaN,NaN,NaN,NaN,2.800000e+01
75%,NaN,NaN,NaN,NaN,NaN,5.600000e+01


#### We could see number of sales is more in year 2017
#### We could see number of sales is more in the month of march
#### We could see number of sales is more in Mumbai(city=4)
#### We could see number of sales is more in the medicine type 453

#### Let us see the same for discount data too

In [50]:
discount["year"]=discount["date"].dt.year# extracting year
discount["month"]=discount["date"].dt.month# extracting month
discount["day"]=discount["date"].dt.day# extracting day

In [51]:
discount.dtypes # checking dtypes

date          datetime64[ns]
city                   int64
medicine               int64
discounted             int64
year                   int64
month                  int64
day                    int64
dtype: object

In [52]:
cols=["city","medicine","year","day","month","discounted"]
for col in cols:
    discount[col]=discount[col].astype("category")

In [53]:
discount.describe(include="all")

,date,city,medicine,discounted,year,month,day
count,17277949,17277949.0,17277949.0,17277949.0,17277949.0,17277949.0,17277949.0
unique,848,10.0,3396.0,2.0,3.0,12.0,31.0
top,2018-02-28 00:00:00,4.0,453.0,0.0,2017.0,7.0,28.0
freq,49607,1809136.0,8490.0,16478900.0,7330001.0,2039107.0,589093.0
first,2016-04-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,2018-07-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


#### We could see a same pattern as that of train data except  number of months is more for july for discounts data

#### Conclusion: so the duplicates are not duplicates as we  have discounts associated for each sale with different price in discounts data too, so , we could sum the sales for each day and also sum the discounts for each day and if there is 2 in discounts column we can replace it with 1 as it is a catagorical column as it is only for  1 day